# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,45.12,93,100,3.44,CA,1697151723
1,1,port-aux-francais,-49.3500,70.2167,36.46,80,83,17.27,TF,1697151950
2,2,adamstown,-25.0660,-130.1015,69.60,69,97,11.83,PN,1697151951
3,3,waitangi,-43.9535,-176.5597,54.82,71,53,5.01,NZ,1697151951
4,4,el salto,20.3333,-104.5000,69.30,79,71,1.90,MX,1697151952


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [21]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    color = "City",
    alpha = 0.5,
    tiles = "OSM"

)

# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
ideal_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 90) & (city_data_df["Max Temp"] > 65) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] < 10) \
    & (city_data_df["Humidity"] < 80)
]

# Drop any rows with null values
ideal_df = ideal_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,vallenar,-28.5708,-70.7581,74.19,24,0,4.23,CL,1697151956
21,21,howard springs,-12.4970,131.0470,85.82,60,9,4.41,AU,1697151959
26,26,guercif,34.2257,-3.3536,70.84,74,1,4.27,MA,1697151961
37,37,tura,25.5198,90.2201,69.55,75,0,2.59,IN,1697151965
95,95,emerald,-23.5333,148.1667,73.35,42,2,2.57,AU,1697151887
130,130,narayangarh,30.4833,77.1333,68.74,53,0,3.02,IN,1697152010
140,140,bulawayo,-20.1500,28.5833,72.41,28,3,2.64,ZW,1697152014
145,145,brigantine,39.4101,-74.3646,65.01,70,0,3.00,US,1697152016
184,184,bharatpur,27.2167,77.4833,77.52,32,0,3.74,IN,1697152034
194,194,remire-montjoly,4.9167,-52.2667,82.44,78,0,2.30,GF,1697152038


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
14,vallenar,CL,-28.5708,-70.7581,24,
21,howard springs,AU,-12.4970,131.0470,60,
26,guercif,MA,34.2257,-3.3536,74,
37,tura,IN,25.5198,90.2201,75,
95,emerald,AU,-23.5333,148.1667,42,
130,narayangarh,IN,30.4833,77.1333,53,
140,bulawayo,ZW,-20.1500,28.5833,28,
145,brigantine,US,39.4101,-74.3646,70,
184,bharatpur,IN,27.2167,77.4833,32,
194,remire-montjoly,GF,4.9167,-52.2667,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":10
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
vallenar - nearest hotel: Hotel del Marqués
howard springs - nearest hotel: No hotel found
guercif - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
emerald - nearest hotel: Emerald Central Hotel
narayangarh - nearest hotel: No hotel found
bulawayo - nearest hotel: Selboune Hotel
brigantine - nearest hotel: La Sammana
bharatpur - nearest hotel: Hotel Jai Palace
remire-montjoly - nearest hotel: Complexe Belova
hamilton - nearest hotel: North Vista Manor
madinat ash shamal - nearest hotel: No hotel found
bandar-e genaveh - nearest hotel: منزل دکتور سلمان
jhalu - nearest hotel: No hotel found
riyadh - nearest hotel: White Palace Hotel
kelibia - nearest hotel: Hôtel La Belle Étoile
francistown - nearest hotel: City Inn
al hazm - nearest hotel: No hotel found
itaueira - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
14,vallenar,CL,-28.5708,-70.7581,24,Hotel del Marqués
21,howard springs,AU,-12.4970,131.0470,60,No hotel found
26,guercif,MA,34.2257,-3.3536,74,No hotel found
37,tura,IN,25.5198,90.2201,75,No hotel found
95,emerald,AU,-23.5333,148.1667,42,Emerald Central Hotel
130,narayangarh,IN,30.4833,77.1333,53,No hotel found
140,bulawayo,ZW,-20.1500,28.5833,28,Selboune Hotel
145,brigantine,US,39.4101,-74.3646,70,La Sammana
184,bharatpur,IN,27.2167,77.4833,32,Hotel Jai Palace
194,remire-montjoly,GF,4.9167,-52.2667,78,Complexe Belova


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)